In [2]:
import numpy as np # linear algebra
import pandas as pd

/var/folders/dn/t8s0kyv52pbfv6q_x3r132rw0000gn/T/ipykernel_21150/3833084863.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
origDF = pd.read_csv('/Users/pranjalmishra/Desktop/python/dataset_ipc.csv',  encoding= 'unicode_escape')

In [4]:
origDF.drop(['Unnamed: 2'], axis = 1, inplace = True)
df = origDF.copy()
df.iloc[320]['description']

'Punishment for culpable homicide not amounting to murder.â\x80\x94Whoever commits culpable\nhomicide not amounting to murder, shall be punished with 1 [imprisonment for life], or imprisonment of\neither description for a term which may extend to ten years, and shall also be liable to fine, if the act by\nwhich the death is caused is done with the intention of causing death, or of causing such bodily injury as\nis likely to cause death; or with imprisonment of either description for a term which may extend to ten years, or with fine, or\nwith both, if the act is done with the knowledge that it is likely to cause death, but without any intention\nto cause death, or to cause such bodily injury as is likely to cause death.'

In [5]:
import re
import nltk
nltk.download('stopwords')

import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS

from collections import Counter

def preprocessingDesc(text):
#     text = re.sub(r"[^a-zA-Z0-9 ]", "", text)
    text = remove_stopwords(text)
    text_en = text.encode("ascii", "ignore")
    text = text_en.decode()
    text = text.replace("[","")
    text = text.replace("]","")
    text = text.replace("\"","")
    text = text.replace("*","")
    text = text.replace("\'","")
#     text = text.replace("\\","")
    text = text.replace("/","")
    text = text.replace("/n","")
    text = text.replace("\n","")
    text = text.replace(";","")
    text = text.lower()
    
#     cnt = Counter()
#     for word in text.split():
#         cnt[word] += 1
    
#     freqwords = set([w for(w,wc) in cnt.most_common(1)])
#     text = ' '.join([word for word in str(text).split() if word not in freqwords])
    
    return text

df['description'] = df['description'].map(lambda s:preprocessingDesc(s))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
ipcs = df['description'].tolist()

In [7]:
from sentence_transformers import SentenceTransformer



In [8]:
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
modelS = SentenceTransformer(model_name)

#Encoding:
sentences_embeddings_a1 = modelS.encode(ipcs)

In [9]:
question = 'punishment if woman die within seven years of marrage'
import nltk
from nltk.util import ngrams
from nltk.metrics.distance  import edit_distance

nltk.download('words')
from nltk.corpus import words
correct_words = words.words()

def autoCorrect(question):
    updated = ""
    check_words = question.split(" ")
    for word in check_words:
        temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
        print(sorted(temp, key = lambda val:val[0])[0][1])
        updated += sorted(temp, key = lambda val:val[0])[0][1] + " "
    return updated
print(question)
question = autoCorrect(question)
print(question)
ques_emb = modelS.encode(question)

[nltk_data] Downloading package words to
[nltk_data]     /Users/pranjalmishra/nltk_data...
[nltk_data]   Package words is already up-to-date!


punishment if woman die within seven years of marrage
punishment
if
woman
die
within
seven
year
of
marriage
punishment if woman die within seven year of marriage 


In [10]:
sentences_embeddings_a1.shape

(553, 768)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
def predict_context(ques_emb, sentences_embeddings_a1):
    count = 0
    sim = 0
    ans = 0

    for emb in sentences_embeddings_a1:
        similar_value_a1 = cosine_similarity([ques_emb], [emb])
        if sim<similar_value_a1:
            ans = count
            sim = similar_value_a1
        count+=1

    print(ans)
    return ans
index = predict_context(ques_emb, sentences_embeddings_a1)

322


In [12]:
print(ipcs[index])
print(origDF.iloc[index]['description'])
res = origDF.iloc[index]['description']

dowry death .(1) where death woman caused burns bodily injury occurs normal circumstances seven years marriage shown soon death subjected cruelty harassment husband relative husband for, connection with, demand dowry, death shall called dowry death, husband relative shall deemed caused death. explanation.for purposes sub-section, dowry shall meaning section 2 dowry prohibition act, 1961 (28 1961). (2) whoever commits dowry death shall punished imprisonment term shall seven years extend imprisonment life.
Dowry death .â(1) Where the death of a woman is caused by any burns or bodily injury or
occurs otherwise than under normal circumstances within seven years of her marriage and it is shown that
soon before her death she was subjected to cruelty or harassment by her husband or any relative of her
husband for, or in connection with, any demand for dowry, such death shall be called âdowry deathâ, and
such husband or relative shall be deemed to have caused her death.
Explanation.âFo

In [13]:
text_en = res.encode("ascii", "ignore")
text = text_en.decode()
answer_text = text.replace("/n","")
answer_text = text.replace("\n"," ")
answer_text

'Dowry death .(1) Where the death of a woman is caused by any burns or bodily injury or occurs otherwise than under normal circumstances within seven years of her marriage and it is shown that soon before her death she was subjected to cruelty or harassment by her husband or any relative of her husband for, or in connection with, any demand for dowry, such death shall be called dowry death, and such husband or relative shall be deemed to have caused her death. Explanation.For the purposes of this sub-section, dowry shall have the same meaning as in section 2 of the Dowry Prohibition Act, 1961 (28 of 1961). (2) Whoever commits dowry death shall be punished with imprisonment for a term which shall not be less than seven years but which may extend to imprisonment for life.]'